# Contour Overlay 

## Imports 
* _numpy_ for array math
* _astropy.io_ for reading and writing FITS cubes and images
* _matplotlib.pyplot_ for plotting spectra and images.

In [ ]:
from astropy.io import fits
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
import numpy as np

import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

## Read in data cube
Read in NIRSpec IFU data cube from Box.   This particular example is a simulated quasar + host galaxy.

In [ ]:
BoxPath='https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/cube_fitting/Q3D_20200407/'
fname='Q3D_NRS_491_s3d.fits'
filename=BoxPath+fname
hdul1=fits.open(filename)
hdul1.info()

with fits.open(filename, memmap=False) as hdulist:
    sci = hdulist["SCI"].data


## Sum over spectral axis

This could be any derivative data product of the same shape as the spatial dimensions of the cube (emission line flux, equivalent width, velocity map, etc...). Here we are doing one of the simplest operations - a sum, for illustrative purposes

In [ ]:
cube_sum=np.sum(sci, axis=0)

## Generate contours
User-specified contour levels in specified units. Custom colors are important, but could be added at a later stage if technically complicated.  Logarithmic and linear spacing options to auto-generate contours would be nice.

In [ ]:
min_level=np.min(cube_sum)
max_level=np.max(cube_sum)
level_colors=['blue', 'purple','red', 'magenta']
contour_levels=[max_level/10000., max_level/1000., max_level/100., max_level/10.]

## Display image and overlay contours

In [ ]:
ax = plt.subplots()[1]

#Image normalization
norm = ImageNormalize(stretch=SqrtStretch())

#Display image
image = ax.imshow(cube_sum, cmap='gray', origin='lower', norm=norm)

#Overlay contours
contour = ax.contour(cube_sum, levels=contour_levels, colors=level_colors)


In [ ]:
contour_levels

In [ ]:
from jdaviz.app import Application
from glue.core import Data
app = Application(configuration='cubeviz')
# data = app.load_data(filename)
app

In [ ]:
app.load_data("/Users/javerbukh/Documents/manga-7495-12704-LOGCUBE_fixed.fits")

In [ ]:
figure_flux = app.get_viewer("flux-viewer")
figure_flux.layers
figure_flux_original_marks = figure_flux.figure.marks

slice_index = figure_flux.state.slices[0]
data = app.data_collection[figure_flux.state.reference_data.label].get_object()
data_at_slice = data.unmasked_data[slice_index,:,:].value

In [ ]:
figure_ivar = app.get_viewer("uncert-viewer")

slice_index_ivar = figure_ivar.state.slices[0]
data_ivar = app.data_collection[figure_ivar.state.reference_data.label].get_object()
data_at_slice_ivar = data_ivar.unmasked_data[slice_index_ivar,:,:].value

In [ ]:
from bqplot import Figure, LinearScale, Axis, ColorScale
from bqplot_image_gl import ImageGL, Contour
import skimage.measure

In [ ]:
min_level=np.min(data_at_slice_ivar)
max_level=np.max(data_at_slice_ivar)
level_colors=['blue', 'purple','red', 'magenta']
contour_levels=[max_level/10000., max_level/1000., max_level/100., max_level/10.]

In [ ]:
contour_list = []
for contour_level in contour_levels:

    contours = skimage.measure.find_contours(data_at_slice_ivar, contour_level)
    contour_list += [k/data_at_slice_ivar.shape for k in contours]
print(contour_levels)
print(contour_list)

In [ ]:
scale_x = LinearScale(min=0, max=1)
scale_y = LinearScale(min=0, max=1)
scales = {'x': scale_x, 'y': scale_y}
axis_x = Axis(scale=scale_x, label='x')
axis_y = Axis(scale=scale_y, label='y', orientation='vertical')
scales_image = {'x': scale_x, 'y': scale_y, 'image': ColorScale(min=np.min(data_at_slice).item(), max=np.max(data_at_slice).item())}
# print(np.min(cube_sum).item(), np.max(cube_sum).item())

figure = Figure(scales=scales, axes=[axis_x, axis_y])
image = ImageGL(image=data_at_slice, scales=scales_image)
contour = Contour(contour_lines=[contour_list], level=contour_levels, scales=scales_image, label_steps=200)
#contour = Contour(image=image, level=contour_levels, scales=scales_image, color="orange")

figure.marks = (image, contour)
figure

In [ ]:
contour.level

In [ ]:
figure_flux.figure.marks = figure_flux.figure.marks + [contour]

In [ ]:
figure_flux.figure.marks = figure_flux_original_marks

Plugin idea: Have the user enter what viewer they would like the contours to appear, what they would like contoured, contour levels, whether it updates with slice changed, and visible bool.



In [ ]:
[x.label for x in app.data_collection.data]

In [ ]:
viewer = app.get_viewer('spectrum-viewer')
[layer_state.layer.label for layer_state in viewer.state.layers]